# **IDENTIFICADOR DE PATOLOGÍAS SÍSMICAS EN EDIFICIOS DE MAMPOSTERIA Y ESTRUCTURAS PORTICADAS DE HORMIGÓN ARMADO**


El objetivo de este proyecto es el de crear un identificador preliminar de patologías edificatorias especializado en sismos.

Éste se centrará en las patologías propias de estructuras de mampostería y estructuras porticadas de hormigón armado por ser éstas algunas de las estructuras más comunes en lugares gravemente afectados por actividad sísmica.

Pretendemos que esta herramienta sirva de apoyo a las personas afectadas por terremotos así como a los profesionales que deberán evaluar el estado de los inmuebles después de dichas catástrofes naturales, pudiendo los últimos disponer así de un apoyo para un rápido diagnóstico. Esperamos que así se pueda agilizar la toma de medidas de seguridad.

Planteamos también en un futuro la posibilidad de trabajar con los datos de las fotografías de usuarios, obteniendo las coordenadas de la fotografía para poder obtener una ubicación de los datos y entender, en caso de sismo, qué zonas han sido más afectadas y necesitan ser priorizadas.

Las patologías con las que se ha entrenado este modelo son las siguientes:


1.  Colapso. Los tipos de colapso más frecuentes que nos encontraremos en esta categoría son:
  - Colapso total
  - Soft story: Falla el piso más débil, muchas veces la planta baja.
  - Pancake: Los suelos colapsan unos sobre otros de manera progresiva.

2.  OOP/Partial collapse (Masonry) : Causado por las fuerzas horizontales perpendiculares al muro.
  - Return wall separation: Separación de un muro de fachada
  - Gable and partial facade collapse: Desplome parcial de elementos de la fachada.
  - Out of plane due to thrust action of horizontal structure (beams).

3.  IP/Partial collapse (Masonry): Causado por las fuerzas horizontales paralelas al muro.
  - Flexural cracking
  - Diagonal cracking
  - Bed joint sliding

4.  Combined IP/OOP and corners (Masonry): Habitualmente las fuerzas se producen en más de una dirección, causando colapsos OOP de elementos que previamente habían sufrido daños IP, o en las esquinas, que son los elementos más frágiles de las construcciones.

5.  RC ST failure (RC): Fallos que ponen en riesgo la capacidad portante de la estructura porticada de hormigón armado.
  - Shear failure of columns
  - Loss of reinforcement
  - Bending or reinforcement
  
6.  Spalling (RC): Pérdida de la capa de recubrimiendo del armado del hormigón o grietas en el mismo. No necesariamente son indicativas de daño estructural y son más fácilmente reparables que las anteriores, aunque deben ser revisadas con cuidado.
   - Spalling
   - Delamination

7.  Infill damage (RC): Daños en los muros de relleno de una estructura porticada. Habitualmente se verán como la separación del muro de los elementos estructurales. Es posible que la estructura no se vea dañada y no presente riesgo inminente de colapso estructural. Sin embargo deben de ser tratados con cuidado dado que el peligro que representan es el desprendimiento total o parcial del muro no estructural, cuya caída es en sí misma un riesgo.
  - OOP damage to infill in concrete structures
  - IP damage to infill in concrete structures

8.  Pounding/Leaning: Daños ocasionados por los movimientos horizontales de un edificio al inclinarse o apoyarse sobre otro.

9.	Toe crushing: falla en la construcción de un muro o columna en el cual los el inferior se agrieta o desmorona.

10.	Dislodgements: Desprendimiento de elementos de acabado.
  - Tiles
  - Bricks in facade
  - Ceiling panels

11.	Healthy: Edificios sin patologías evidentes.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomFlip, RandomRotation, RandomZoom

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_img_Path = '/content/drive/MyDrive/Patologías Edificatorias/TEST_TRAIN_SPLIT/TRAIN'
test_img_Path= '/content/drive/MyDrive/Patologías Edificatorias/TEST_TRAIN_SPLIT/TEST'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageEnhance

#Vamos a crear una función que recorte nuestras imagenes como un cuadrado

def recortar_imagen(imagen):
    alto, ancho = tf.shape(imagen)[0], tf.shape(imagen)[1]
    lado_mas_corto = tf.minimum(alto, ancho)

    # Calcular las coordenadas de recorte para centrar la imagen
    inicio_y = (alto - lado_mas_corto) // 2
    inicio_x = (ancho - lado_mas_corto) // 2

    # Recortar la imagen
    imagen_recortada = tf.image.crop_to_bounding_box(imagen, inicio_y, inicio_x, lado_mas_corto, lado_mas_corto)

    # Redimensionar la imagen al nuevo tamaño
    imagen_recortada = tf.image.resize(imagen_recortada, (224, 224))

    return imagen_recortada

In [ ]:
#vamos a normalizar los datos de nuestras imágenes. Cada canal toma un valor del 0 al 255, por lo qoue para escalar habremos de dividir por ese valor.

datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=recortar_imagen,
)

In [ ]:
#Creamos un flujo de datos de las imagenes recortadas segun el datagen anterior, manteniendo la estructura de carpetas (etiquetas)

data_generator = datagen.flow_from_directory(
    train_img_Path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=True
)

Found 7275 images belonging to 11 classes.


In [ ]:
#hemos escogido un batch size de 50 ya que nuestro dataset no es demasiado grande. El tamaño de nuestras imagenes es de 224. Con este tamaño de imagen fue entrenada la red ResNet50 que hemos elegido para nuestro modelo.

image_size = (224, 224)
batch_size = 50

# A continuación creamos un dataset para entrenamiento y otro para validación usando keras

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
   train_img_Path,
   seed=123,
   validation_split=0.25,
   subset="training",
   image_size=image_size,
   batch_size=batch_size
)
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
   test_img_Path,
   seed=123,
   validation_split=0.25,
   subset="validation",
   image_size=image_size,
   batch_size=batch_size
)


Found 7275 files belonging to 11 classes.
Using 5457 files for training.
Found 1803 files belonging to 11 classes.
Using 450 files for validation.


In [ ]:
#Hacemos una comprobación de que todas las clases han sido leídas correctamente

print(train_ds.class_names)
print(validation_ds.class_names)

['01_COLLAPSE', '02_OOP_PARTIAL COLLAPSE', '03_IP_PARTIAL COLLAPSE', '04_COMBINED_CORNERS', '05_RC ST FAILURE', '06_SPALLING', '07_DAMAGE TO INFILL', '08_POUNDING_LEANING', '09_TOE CRUSHING', '10_DISLODGEMENTS', '11_HEALTHY']
['01_COLLAPSE', '02_OOP_PARTIAL COLLAPSE', '03_IP_PARTIAL COLLAPSE', '04_COMBINED_CORNERS', '05_RC ST FAILURE', '06_SPALLING', '07_DAMAGE TO INFILL', '08_POUNDING_LEANING', '09_TOE CRUSHING', '10_DISLODGEMENTS', '11_HEALTHY']


In [ ]:
# Importamos las librerías de keras
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Activation
from keras.layers.experimental import preprocessing


In [ ]:
# Importamos el modelo preentrenado. Hemos elegido ResNet50, una red con 50 capas de profundidad entrenada con objetos genéricos.

imported_model = tf.keras.applications.ResNet50(
    # Nos aseguramos de poder incluir capas de keras para nuestro input y output.
    include_top=False,
    # Especificamos las dimensiones del input y los canales, que serán 3 (RGB)
    input_shape=(224,224,3),
    pooling=None, # Para que devuelva 4D
    # Utilizaremos los pesos disponibles en imagenet.
    weights='imagenet'
    )

for layer in imported_model.layers:
  # Nos aseguramos de que los parámetros y pesos no se sometan a un nuevo entrenamiento, así aceleramos el proceso de aprendizaje.
  layer.trainable=False

In [ ]:
# Inicializamos un modelo secuencial para acceder a ResNet50
dnn_model = Sequential()

In [ ]:
#importamos los comandos específicos que usaremos para crear nuestra red neuronal convolucional y para tratar de mitigar el overfitting.

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# Añadimos ResNet50 como primera capa de nuestro modelo.
dnn_model.add(imported_model)

# Añadimos nuestras propias capas con regularizacion L2, Dropout y BatchNormalization para tratar de evitar overfitting.

dnn_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
dnn_model.add(MaxPooling2D((2, 2)))
dnn_model.add(BatchNormalization())
dnn_model.add(Dropout(0.3))

dnn_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
dnn_model.add(MaxPooling2D((2, 2)))
dnn_model.add(BatchNormalization())
dnn_model.add(Dropout(0.3))

dnn_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# dnn_model.add(MaxPooling2D((2, 2)))
dnn_model.add(BatchNormalization())
dnn_model.add(Dropout(0.4))

dnn_model.add(Flatten())
dnn_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
dnn_model.add(Dropout(0.5))

dnn_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
dnn_model.add(Dropout(0.5))

# Añadimos finalmente una capa densa con función de activación softmax. Esta capa tendrá tantas neuronas como categorías queramos identificar (11 en nuestro caso)
dnn_model.add(Dense(11, activation='softmax'))

In [ ]:
# Compilamos el modelo con un optiizador adam

opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    name='adam',
)

dnn_model.compile(
    optimizer= opt,
    # Utilizamos las pérdidas para calcular los errores del modelo.
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    # Añadimos métricas para calcular el desempeño de nuestro modelo
    metrics=['accuracy']
    )

In [ ]:
#Vamos a crear un earlystopping. Para ello vamos a monitorear el las pérdidas de nuestro set de validación, de tal manera que si no mejora en X epocas, nuestro modelo dejará de entrenarse para evitar el overfitting.
#Pare donde pare, se quedará con los mejores datos obtenidos para val_loss.

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)


In [ ]:
# Entrenamos nuestro modelo con los datasets que creamos para entrenamiento y validación.

history = dnn_model.fit(
train_ds,
validation_data=validation_ds,
epochs=25,
callbacks=[early_stopping]
)

Epoch 1/25
110/110 [==============================] - 1516s 14s/step - loss: 2.7515 - accuracy: 0.1209 - val_loss: 2.4188 - val_accuracy: 0.2111
Epoch 2/25
110/110 [==============================] - 1415s 13s/step - loss: 2.3520 - accuracy: 0.2021 - val_loss: 2.1216 - val_accuracy: 0.3356
Epoch 3/25
110/110 [==============================] - 1419s 13s/step - loss: 2.1090 - accuracy: 0.2795 - val_loss: 1.8208 - val_accuracy: 0.4267
Epoch 4/25
 61/110 [===============>..............] - ETA: 9:52 - loss: 1.9220 - accuracy: 0.3416 

In [ ]:
# Vamos a ver las gráficas de nuestro modelo.

# Obtenemos los valores de pérdida y precisión del entrenamiento
import matplotlib.pyplot as plt
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Crear una figura
plt.figure(figsize=(12, 4))

# Subplot para la pérdida
plt.subplot(1, 2, 1)
plt.plot(history.epoch, train_loss, label='Training Loss')
plt.plot(history.epoch, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

# Subplot para la precisión
plt.subplot(1, 2, 2)
plt.plot(history.epoch, train_accuracy, label='Training Accuracy')
plt.plot(history.epoch, val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

# Mostrar la figura
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

#Vamos a hacer alguna prueba con nuestras propias imágenes.

# Ruta a la imagen que quieres probar
image_path = '/content/drive/MyDrive/SATURDAYS AI/DATA/MY_VAL/RC.jpg'

# Cargamos y preprocesamos la imagen
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión para el batch (4D)

# Visualizamos la imagen de entrada
plt.imshow(img_array[0]/255) #Normalización
plt.title('Imagen de entrada')
plt.axis('on')
plt.show()

# Hacemos la predicción
predictions = dnn_model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Mapeamos el índice de clase a la etiqueta obtenida
class_names = ['01_COLLAPSE', '02_OOP_PARTIAL COLLAPSE', '03_IP_PARTIAL COLLAPSE', '04_COMBINED_CORNERS', '05_RC ST FAILURE', '06_SPALLING', '07_DAMAGE TO INFILL', '08_POUNDING_LEANING', '09_TOE CRUSHING', '10_DISLODGEMENTS', '11_HEALTHY']  # Asegúrate de que las etiquetas están en el orden correcto
predicted_label = class_names[predicted_class[0]]

# Imprimimos todas las probabilidades
print("Probabilidades de cada clase:")
for i, prob in enumerate(predictions[0]):
    print(f'{class_names[i]}: {prob:.4f}')

print(f'Predicción: {predicted_label}')

In [ ]:
#Vamos a proceder a la exportación del modelo y los pesos para nuestra API.

import sys
import os

dir = '/content/drive/MyDrive/SATURDAYS AI/DATA/MODELO4'

if not os.path.exists(dir):
  os.mkdir(dir)

dnn_model.save('/content/drive/MyDrive/SATURDAYS AI/DATA/MODELO4/modelo.h5')
dnn_model.save_weights('/content/drive/MyDrive/SATURDAYS AI/DATA/MODELO4/pesos.h5')

In [ ]:
# Save the history to a file (e.g., using pickle)
import pickle
history_path='/content/drive/MyDrive/SATURDAYS AI/DATA/MODELO4/model_history.pkl'

with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)